In [1]:
import os , sys
import pandas as pd
from pathlib import Path
import numpy as np
from pydantic import BaseModel, field_validator, ValidationError


In [2]:
pkg_pth = Path.cwd().parent

print(pkg_pth)

sys.path.append(str(pkg_pth))
print(sys.path)

/home/t/atest/churn-prediction-mlops
['/home/t/miniconda3/lib/python312.zip', '/home/t/miniconda3/lib/python3.12', '/home/t/miniconda3/lib/python3.12/lib-dynload', '', '/home/t/atest/churn-prediction-mlops/.venv/lib/python3.12/site-packages', '/home/t/atest/churn-prediction-mlops']


In [3]:
df = pd.read_csv('../dataset/data.csv')

In [4]:
df.shape

(10000, 14)

In [5]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [7]:
for i in df.columns:
    print(i, df[i].dtype)

RowNumber int64
CustomerId int64
Surname object
CreditScore int64
Geography object
Gender object
Age int64
Tenure int64
Balance float64
NumOfProducts int64
HasCrCard int64
IsActiveMember int64
EstimatedSalary float64
Exited int64


In [8]:
df['Geography'].unique()

array(['France', 'Spain', 'Germany'], dtype=object)

In [9]:
from src.data_validation import Customer, schema_validation

In [10]:

dtypes = {
    'integer': [np.dtypes.Int64DType, np.dtypes.Int32DType, np.dtypes.Int16DType, np.dtypes.Int8DType, np.dtypes.UInt64DType, np.dtypes.UInt32DType, np.dtypes.UInt16DType, np.dtypes.UInt8DType],
    'number': [float, np.dtypes.Float16DType, np.dtypes.Float32DType, np.dtypes.Float64DType],
    'string': [str, np.dtypes.ObjectDType , np.dtypes.StringDType],
    'boolean': [bool, np.dtypes.BoolDType],
    'datetime': [np.dtypes.TimeDelta64DType],
    'categorical': [pd.CategoricalDtype],
}

def map_type(x):
    for key, value in dtypes.items():
        if x in value:
            return key

        

In [ ]:
def schema_validation(csv_filepath: str, nrows: int = None) -> bool:
    try:
        df = pd.read_csv(csv_filepath, nrows=nrows) 

        fs = {col : map_type(type(df[col].dtype)) for col in df.columns}
        pp = Customer.model_json_schema()['properties']
        ss = {k:v['type'] for k,v in pp.items()}

        for i in pp:
            if not  fs[i]==ss[i] :
                assert False, f"Type mismatch for column {i}"
        
        return True
    
    except (ValidationError, pd.errors.ParserError, ValueError, AssertionError) as e:  
        print(f"Validation Error: {e}")
        return False


In [19]:
schema_validation('../dataset/sample_data.csv')

Validation Error: Type mismatch for column CreditScore


False

In [15]:
Customer.model_json_schema()

{'properties': {'RowNumber': {'title': 'Rownumber', 'type': 'integer'},
  'CustomerId': {'title': 'Customerid', 'type': 'integer'},
  'Surname': {'title': 'Surname', 'type': 'string'},
  'CreditScore': {'title': 'Creditscore', 'type': 'integer'},
  'Geography': {'title': 'Geography', 'type': 'string'},
  'Gender': {'title': 'Gender', 'type': 'string'},
  'Age': {'title': 'Age', 'type': 'integer'},
  'Tenure': {'title': 'Tenure', 'type': 'integer'},
  'Balance': {'title': 'Balance', 'type': 'number'},
  'NumOfProducts': {'title': 'Numofproducts', 'type': 'integer'},
  'HasCrCard': {'title': 'Hascrcard', 'type': 'integer'},
  'IsActiveMember': {'title': 'Isactivemember', 'type': 'integer'},
  'EstimatedSalary': {'title': 'Estimatedsalary', 'type': 'number'},
  'Exited': {'title': 'Exited', 'type': 'integer'}},
 'required': ['RowNumber',
  'CustomerId',
  'Surname',
  'CreditScore',
  'Geography',
  'Gender',
  'Age',
  'Tenure',
  'Balance',
  'NumOfProducts',
  'HasCrCard',
  'IsActiveM

In [ ]:

fs

{'RowNumber': 'integer',
 'CustomerId': 'integer',
 'Surname': 'string',
 'CreditScore': 'integer',
 'Geography': 'string',
 'Gender': 'string',
 'Age': 'integer',
 'Tenure': 'integer',
 'Balance': 'number',
 'NumOfProducts': 'integer',
 'HasCrCard': 'integer',
 'IsActiveMember': 'integer',
 'EstimatedSalary': 'number',
 'Exited': 'integer'}